# Лекция 3: Классификация и ансамблевые методы

## Классификация

Задача классификации заключается в том, чтобы по входным данным $\mathbf{x} \in \mathbb{R}^d$ предсказать метку класса $y$, которая принадлежит конечному множеству категорий, например,
$$
y \in \{0, 1\} \quad \text{(бинарная классификация)},
$$
или
$$
y \in \{1, 2, \dots, K\} \quad \text{(многоклассовая классификация)}.
$$

Цель обучения заключается в построении модели $f: \mathbb{R}^d \to \text{Labels}$, которая минимизирует некоторую функцию потерь между истинными метками и предсказаниями.

### Логистическая регрессия

#### Постановка задачи

Пусть $y \in \{0, 1\}$, а вектор признаков $\mathbf{x} \in \mathbb{R}^d$. Модель логистической регрессии задаётся следующим образом:
$$
P(y=1 \mid \mathbf{x}) = \sigma(z) = \frac{1}{1 + e^{-z}}, \quad \text{где } z = \mathbf{w}^\top \mathbf{x} + b.
$$
Соответственно,
$$
P(y=0 \mid \mathbf{x}) = 1 - \sigma(\mathbf{w}^\top \mathbf{x} + b).
$$

#### Функция правдоподобия и оптимизация

Для определения параметров $\mathbf{w}$ и $b$ используется метод максимального правдоподобия. Правдоподобие всей выборки:
$$
\mathcal{L}(\mathbf{w}, b) = \prod_{i=1}^N \sigma(\mathbf{w}^\top \mathbf{x}_i + b)^{y_i} \left[1 - \sigma(\mathbf{w}^\top \mathbf{x}_i + b)\right]^{1-y_i}.
$$
Переходя к логарифмическому правдоподобию, получаем:
$$
\ell(\mathbf{w}, b) = \sum_{i=1}^N \left[ y_i \ln \sigma(\mathbf{w}^\top \mathbf{x}_i + b) + (1-y_i) \ln \left(1-\sigma(\mathbf{w}^\top \mathbf{x}_i + b)\right) \right].
$$
Оптимизационная задача сводится к поиску параметров:
$$
\max_{\mathbf{w},\,b} \; \ell(\mathbf{w}, b),
$$
или эквивалентно – к минимизации отрицательного логарифмического правдоподобия (кросс-энтропийной функции потерь).

#### Особенности

- **Линейная граница разделения:** Модель определяет гиперплоскость, которая делит пространство на две области.
- **Использование градиентного спуска:** В аналитическом виде задача не решается, поэтому применяются итерационные методы оптимизации (градиентный спуск, Ньютон и др.).

<img src="https://lh7-rt.googleusercontent.com/docsz/AD_4nXcHuZ2ujpLJBtvr6h_R3W_yR10Ao4K-SDGEWcs00BX9nriJD7RJisOPIWyOzbLaaIDSTvqhwZaSm-krhGfdlsby-DWyFTNrPzUpU1M0g4-FluB83uNt5e9iBBAuz80pUmBP_hMwTlraiFhOLlhxmamxqHB0?key=3-PO9Abu6TW8hu9LQuqfMg" alt="Описание изображения" width="600">

In [13]:
import numpy as np
import matplotlib.pyplot as plt

def plot_decision_boundary(model, X, y, ax=None, cmap="coolwarm", h=0.02):
    """
    Строит график разделяющей границы для модели классификации.

    Аргументы:
        model (sklearn-модель): обученная модель классификации.
        X (np.ndarray): массив признаков, размер (n_samples, 2).
        y (np.ndarray): массив целевых меток.
        ax (matplotlib.axes.Axes, optional): ось для построения графика.
        cmap (str): название цветовой карты.
        h (float): шаг для создания сетки.
        
    Возвращает:
        ax (matplotlib.axes.Axes): ось с графиком разделяющей границы.
    """
    if ax is None:
        ax = plt.gca()
    
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap=cmap)
    ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors="k", cmap=cmap)
    
    ax.set_xlabel("Признак 1")
    ax.set_ylabel("Признак 2")
    ax.set_title("Разделяющая граница логистической регрессии")
    
    return ax

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, 
                             confusion_matrix, 
                             classification_report)


np.random.seed(42)
X, y = make_classification(n_samples=50,
                            n_features=2,
                            n_redundant=0,
                            n_informative=2,
                            n_clusters_per_class=1,
                            flip_y=0.1,
                            class_sep=1.5,
                            random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=42)

log_reg = LogisticRegression(solver="lbfgs")
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred))
print("Отчет по классификации:\n", classification_report(y_test, y_pred))

plt.figure(figsize=(8, 6))
plot_decision_boundary(log_reg, X, y)
plt.tight_layout()
plt.show()

### Деревья решений

#### Принцип работы

1. **Корневой узел:** Всю выборку $D$ рассматривают как набор в корневом узле.
2. **Разбиение:** На каждом шаге выбирается признак и порог, по которому выборка разделяется на два (или более) подмножеств $D_1$ и $D_2$. Критерий выбора разбиения основан на уменьшении нечистоты узла.
3. **Остановка:** Процесс рекурсивного деления продолжается до достижения определённых условий (максимальная глубина, минимальное число объектов в узле, достижение чистого узла и т.д.).

#### Метрики для разбиения

##### Энтропия и информационный выигрыш

Энтропия узла $D$ определяется как:
$$
H(D) = -\sum_{k} p_k \log_2 p_k,
$$
где $p_k$ — доля объектов, принадлежащих классу $k$. При разбиении узла информационный выигрыш ($IG$) вычисляется по формуле:
$$
IG(D, A) = H(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} H(D_v),
$$
где $D_v$ — подмножество данных, для которых значение признака $A$ равно $v$.

##### Индекс Джини

Другой распространённый критерий нечистоты – индекс Джини:
$$
G(D) = 1 - \sum_{k} p_k^2.
$$
При выборе разбиения ищут признак, который даёт максимальное уменьшение индекса Джини:
$$
\Delta G = G(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} G(D_v).
$$

#### Особенности

- **Жадный алгоритм:** На каждом шаге выбирается локально оптимальное разбиение, что не гарантирует глобально оптимального дерева.
- **Переобучение:** Очень глубокие деревья могут переобучаться, поэтому применяются методы отсечения (pruning) для улучшения обобщающей способности модели.
- **Интерпретируемость:** Деревья решений являются интуитивно понятными и наглядно демонстрируют процесс принятия решений.

<img src="https://help.pyramidanalytics.com/Content/Root/MainClient/apps/Model/Model%20Pro/Data%20Flow/ML/Images/DecisionTrees-MachineLearining-01.png" alt="Описание изображения" width="500">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


np.random.seed(42)
X, y = make_classification(n_samples=50,
                            n_features=2,
                            n_redundant=0,
                            n_informative=2,
                            n_clusters_per_class=1,
                            flip_y=0.1,
                            class_sep=1.5,
                            random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

y_pred = tree_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred))
print("Отчет по классификации:\n", classification_report(y_test, y_pred))

plt.figure(figsize=(8, 6))
plot_decision_boundary(tree_clf, X, y)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 8))
plot_tree(tree_clf,
            feature_names=["Признак 1", "Признак 2"],
            filled=True,
            rounded=True,
            fontsize=10)
plt.title("Графическое представление дерева решений")
plt.tight_layout()
plt.show()

### Bias variance decomposition

Одним из центральных понятий в машинном обучении является **разложение ошибки на смещение и дисперсию (bias-variance decomposition)**. Оно позволяет понять, какие составляющие вносят вклад в общую ошибку модели и как можно управлять этим балансом при выборе и настройке моделей.

#### Теория
Пусть у нас имеется функция истинной зависимости $ y = f(x) + \varepsilon $, где:
- $ f(x) $ – истинная функция,
- $\varepsilon$ – случайный шум с дисперсией $\sigma^2$ (неизбежная ошибка, относящаяся к шуму).

Общая квадратная ошибка модели $\hat{f}(x)$ для фиксированной точки $x$ определяется как:
$$
\mathbb{E}\left[(y - \hat{f}(x))^2\right] = \underbrace{\left( \mathbb{E}[\hat{f}(x)] - f(x) \right)^2}_{\text{Bias}^2} + \underbrace{\mathbb{E}\left[\left(\hat{f}(x) - \mathbb{E}[\hat{f}(x)]\right)^2\right]}_{\text{Variance}} + \sigma^2.
$$

- **Смещение (Bias):** Показывает, насколько среднее предсказание модели отклоняется от истинной функции $ f(x) $. Высокое смещение может возникать, если модель слишком проста и не способна уловить сложность зависимости (так называемое недообучение).

- **Дисперсия (Variance):** Характеризует изменчивость предсказаний модели при изменении обучающей выборки. Высокая дисперсия свойственна сложным моделям, которые сильно адаптируются к обучающим данным (что может привести к переобучению).

Баланс между bias и variance определяет качество обобщения модели. Идеальной моделью является такая, которая имеет как низкое смещение, так и низкую дисперсию, однако в реальных задачах часто наблюдается компромисс между ними.

#### Сильные и слабые модели
В контексте ансамблирования модели часто делят на **слабые** и **сильные**:

- **Слабые модели** – это модели, которые лишь немного лучше случайного угадывания. В задачах классификации такие алгоритмы называют «слабым обучением». Пример: *решающее дерево с одной глубиной* (decision stump) или в целом неглубокие деревья. Они, как правило, имеют высокое смещение (не могут уловить сложные зависимости), но обладают низкой дисперсией.

- **Сильные модели** – это более сложные модели с хорошей способностью описывать данные (низкое смещение), но зачастую они обладают высокой дисперсией. Пример: полные, глубоко растущие деревья решений без отсечения. В ансамблировании, например, случайный лес использует именно такие сильные модели, и за счёт бутстреп-агрегации (bagging) достигается значительное снижение дисперсии.

Различные методы ансамблирования используют именно эти особенности:

- **Бэггинг (Bagging):**  
  Обычно применяется для сильных моделей с высоким разбросом (variance). Например, полные деревья решений могут сильно колебаться от разбиения к разбиению, и их усреднение снижает дисперсию предсказания.

- **Бустинг (Boosting):**
  Часто использует слабые модели с ограниченной глубиной (низкая дисперсия, высокое смещение), внося последовательные коррективы в модель. Каждая последующая слабая модель обучается на исправлении ошибок предыдущих, что позволяет снижать смещение итогового ансамбля.


<img src="https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/images/bias_variance/bullseye.png" alt="Описание изображения" width="500">

<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2024/07/eba93f5a75070f0fbb9d86bec8a009e9.webp" alt="Описание изображения" width="500">

In [29]:
!pip install mlxtend seaborn -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from mlxtend.evaluate import bias_variance_decomp

np.random.seed(42)
X, y = make_moons(n_samples=500, noise=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1)

clf = DecisionTreeClassifier(random_state=1)
avg_error, avg_bias, avg_var = bias_variance_decomp(clf, 
                                                    X_train, y_train, 
                                                    X_test, y_test,
                                                    loss='0-1_loss', 
                                                    num_rounds=200, 
                                                    random_seed=1)

print("Average Classification Error: {:.3f}".format(avg_error))
print("Average Bias: {:.3f}".format(avg_bias))
print("Average Variance: {:.3f}".format(avg_var))

labels = ['Error', 'Bias', 'Variance']
values = [avg_error, avg_bias, avg_var]

plt.figure(figsize=(8, 6))
colors = sns.color_palette("Set2", n_colors=len(labels))
bars = plt.bar(labels, values, color=colors)
plt.ylabel("Значение")
plt.title("Разложение ошибки (Bias-Variance) для Decision Tree Classifier")
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, 
             f"{yval:.3f}", ha='center', va='bottom')
plt.tight_layout()
plt.show()

### Ансамблевые методы

Ансамблевые методы объединяют несколько моделей для получения более точного и стабильного результата. Они используются для уменьшения дисперсии, смещения или общей ошибки модели.

<img src="https://cdn.corporatefinanceinstitute.com/assets/ensemble-methods.png" alt="Описание изображения" width="600">

<img src="https://intuitivetutorial.com/wp-content/uploads/2023/05/ensemble_models.png" alt="Описание изображения" width="600">

In [63]:
import numpy as np

def true_function(x):
    """
    Истинная функция: sin(x)
    """
    return np.sin(x)


def run_experiments(model, n_train, noise_std, x_grid, n_experiments):
    """
    Запускает серию экспериментов для оценки предсказаний модели.

    Аргументы:
        model: экземпляр модели (например, DecisionTreeRegressor или RandomForestRegressor)
        n_train (int): число обучающих образцов в каждом эксперименте.
        noise_std (float): стандартное отклонение шума.
        x_grid (np.ndarray): массив точек (форма (n_points, 1)) для вычисления предсказаний.
        n_experiments (int): количество экспериментов.

    Возвращает:
        predictions (np.ndarray): массив предсказаний формы (n_experiments, n_points).
    """
    predictions = []
    for _ in range(n_experiments):
        X_train = np.random.uniform(0, 2*np.pi, n_train).reshape(-1, 1)
        noise = np.random.normal(0, noise_std, n_train)
        y_train = true_function(X_train.ravel()) + noise

        model.fit(X_train, y_train)
        pred = model.predict(x_grid)
        predictions.append(pred)
    return np.array(predictions)


def bias_variance_decomposition(predictions, y_true):
    """
    Вычисляет bias² и variance для набора предсказаний по координатам.

    Аргументы:
        predictions (np.ndarray): матрица предсказаний формы (n_experiments, n_points).
        y_true (np.ndarray): истинные значения функции для точек сетки (n_points,).

    Возвращает:
        global_bias (float): усреднённое значение bias² по всем точкам сетки.
        global_variance (float): усреднённое значение дисперсии предсказаний.
    """
    mean_pred = np.mean(predictions, axis=0)
    bias_sq = (y_true - mean_pred)**2
    variance = np.var(predictions, axis=0)
    global_bias = np.mean(bias_sq)
    global_variance = np.mean(variance)
    return global_bias, global_variance


def display_results(model_names, bias_values, variance_values):
    """
    Выводит результаты (bias² и variance для каждой модели) и строит группированный столбчатый график.

    Аргументы:
        model_names (list[str]): список имен моделей.
        bias_values (list[float]): список значений bias² для каждой модели.
        variance_values (list[float]): список значений variance для каждой модели.
    """
    for name, bias, var in zip(model_names, bias_values, variance_values):
        print(f"{name}:")
        print(f"  Bias²: {bias:.3f}, Variance: {var:.3f}\n")

    _, ax = plt.subplots(figsize=(6, 4))
    width = 0.35
    indices = np.arange(len(model_names))

    palette = sns.color_palette("Set2", n_colors=2)
    bars_bias = ax.bar(indices - width / 2, bias_values, width, 
                       label="Bias²", color=palette[0])
    bars_variance = ax.bar(indices + width / 2, variance_values, width, 
                           label="Variance", color=palette[1])

    ax.set_xlabel("Модель")
    ax.set_ylabel("Значение")
    ax.set_xticks(indices)
    ax.set_xticklabels(model_names)
    ax.legend()

    for bar in bars_bias + bars_variance:
        height = bar.get_height()
        ax.annotate(f"{height:.3f}",
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha="center", va="bottom")

    plt.tight_layout()
    plt.show()

#### Bagging

**Bootstrap Aggregating (Bagging)** заключается в обучении множества моделей на различных бутстреп-выборках из исходного набора данных. Итоговый прогноз вычисляется как агрегированное (например, голосование для классификации) предсказание отдельных 
моделей.

**Важно!** утстрепная выборка имеет такой же размер, что и исходная (генерация с повторениями)

Пусть $ D $ — исходная выборка из $ N $ объектов. Из неё генерируются $ M $ бутстреп-выборок $ D^{(1)}, D^{(2)}, \ldots, D^{(M)} $. Для каждой выборки $ D^{(m)} $ обучается модель $ f^{(m)}(x) $. Тогда итоговая модель имеет вид:
  $$
  f_{\text{bag}}(x) = \frac{1}{M}\sum_{m=1}^{M} f^{(m)}(x).
  $$
  Для задач классификации итоговое решение часто определяется по принципу большинства голосов.

**Особенность:** Снижает дисперсию модели, стабилизирует предсказания и уменьшает риск переобучения.

**Пример:**
    Random Forest – ансамбль деревьев решений, где каждая модель обучается на бутстреп-выборке, а при выборе разбиений учитывается случайное подмножество признаков.

- Каждое дерево обучается на случайной бутстреп-выборке исходных данных.
- При построении каждого дерева на каждом узле выбирается случайное подмножество признаков для разбиения, что снижает корреляцию между деревьями.

Для $ M $ деревьев решений, где $ f^{(m)}(x) $ – предсказание $ m $-го дерева, итоговое предсказание определяется как:
$$
f_{\text{RF}}(x) = \frac{1}{M}\sum_{m=1}^{M} f^{(m)}(x),
$$
при этом для задач классификации применяется голосование, а для регрессии – усреднение.

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20230731175958/Bagging-classifier.png" alt="Описание изображения" width="600">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


np.random.seed(42)
n_train = 20
noise_std = 0.3
n_experiments = 100
x_grid = np.linspace(0, 2*np.pi, 100).reshape(-1, 1)
y_true = true_function(x_grid.ravel())

tree_model = DecisionTreeRegressor()
tree_predictions = run_experiments(tree_model, n_train, 
                                   noise_std, x_grid, n_experiments)
tree_bias, tree_variance = bias_variance_decomposition(tree_predictions, y_true)

rf_model = RandomForestRegressor(n_estimators=100)
rf_predictions = run_experiments(rf_model, n_train, 
                                 noise_std, x_grid, n_experiments)
rf_bias, rf_variance = bias_variance_decomposition(rf_predictions, y_true)

model_names = ["Decision Tree", "Bagging"]
bias_values = [tree_bias, rf_bias]
variance_values = [tree_variance, rf_variance]

display_results(model_names, bias_values, variance_values)

#### Boosting

Boosting – метод, в котором модели обучаются последовательно, каждая следующая модель сосредотачивается на ошибках предыдущих.

**Основная идея:**
    1. Изначально каждому объекту присваивается вес:  
        $$
        w_i^{(1)} = \frac{1}{N}, \quad i=1,\dots,N.
        $$
    2. На каждом шаге $ m $ обучается слабый классификатор $ h_m(x) $ согласно текущей взвешенной выборке. Ошибка классификации определяется как:
        $$
        \epsilon_m = \sum_{i=1}^{N} w_i^{(m)} \mathbb{I}\left( y_i \neq h_m(x_i) \right).
        $$
    3. Вычисляется вес модели:
        $$
        \gamma_m = \ln\frac{1-\epsilon_m}{\epsilon_m}.
        $$
    4. Обновление весов объектов производится по формуле:
        $$
        w_i^{(m+1)} = w_i^{(m)} \exp\left( \gamma_m \mathbb{I}\left( y_i \neq h_m(x_i) \right) \right),
        $$
        с последующей нормировкой так, чтобы $\sum_i w_i^{(m+1)} = 1$.

**Особенность:** Снижает смещение модели, позволяет добиться высокой точности за счет концентрации на сложных для классификации объектах.

**Пример:** 
**Gradient Boosting**, где последовательно обучаются модели для минимизации остаточной ошибки с использованием градиентного спуска.
Каждое новое базовое дерево обучается на остатках (или псевдопроизводных) функции потерь предыдущего ансамбля.

Итоговая модель представлена как:
$$
f(x) = \sum_{m=1}^{M} \gamma_m h_m(x).
$$
На шаге $ m $ новая модель $ h_m(x) $ решает задачу аппроксимации негативного градиента:
$$
r_i^{(m)} = -\left.\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right|_{f(x) = f_{m-1}(x)},
$$
а коэффициент $ \gamma_m $ выбирается для минимизации:
$$
\gamma_m = \arg\min_{\gamma} \sum_{i=1}^{N} L\left( y_i, f_{m-1}(x_i) + \gamma\, h_m(x_i) \right).
$$
Здесь $ L(y, f(x)) $ – функция потерь, например, логарифмическая потеря для классификации или квадратичная ошибка для регрессии.

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20210707140911/Boosting.png" alt="Описание изображения" width="600">

In [65]:
!pip install xgboost -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor


np.random.seed(42)
n_train = 20
noise_std = 0.3
n_experiments = 100
x_grid = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_true = true_function(x_grid.ravel())

tree_model = DecisionTreeRegressor(random_state=42)
tree_predictions = run_experiments(tree_model, n_train, 
                                   noise_std, x_grid, n_experiments)
tree_bias, tree_variance = bias_variance_decomposition(tree_predictions, y_true)

xgb_model = XGBRegressor(n_estimators=100,
                         max_depth=2,
                         learning_rate=0.05,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         random_state=42,
                         objective="reg:squarederror",
                         verbosity=0)

xgb_predictions = run_experiments(xgb_model, n_train, 
                                  noise_std, x_grid, n_experiments)
xgb_bias, xgb_variance = bias_variance_decomposition(xgb_predictions, y_true)

model_names = ["Decision Tree", "Boosting"]
bias_values = [tree_bias, xgb_bias]
variance_values = [tree_variance, xgb_variance]

display_results(model_names, bias_values, variance_values)

### Stacking

Метод стекинга объединяет прогнозы нескольких базовых моделей посредством мета-модели. Базовые модели (уровень-0) выдают предсказания, которые затем используются в качестве входных признаков для мета-модели (уровень-1). Итоговая модель может быть записана как:
  $$
  f_{\text{stack}}(x) = g\Bigl( h_1(x), h_2(x), \dots, h_M(x) \Bigr),
  $$
  где $ g $ – обучаемая мета-функция.

**Особенность:** Позволяет объединить сильные стороны различных моделей, что зачастую приводит к улучшению качества предсказаний.

<img src="https://www.appliedaicourse.com/blog/wp-content/uploads/2024/10/architecture-of-a-stacking-model-1024x534.jpg" alt="Описание изображения" width="600">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import StackingRegressor


np.random.seed(42)
n_train = 20
noise_std = 0.3
n_experiments = 100
x_grid = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_true = true_function(x_grid.ravel())

tree_model = DecisionTreeRegressor(random_state=42)
tree_predictions = run_experiments(tree_model, n_train, noise_std, x_grid, n_experiments)
tree_bias, tree_variance = bias_variance_decomposition(tree_predictions, y_true)

estimators = [
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('ridge', Ridge())
]

stack_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(),
    cv=5,
    n_jobs=-1
)

stack_predictions = run_experiments(stack_model, n_train, noise_std, x_grid, n_experiments)
stack_bias, stack_variance = bias_variance_decomposition(stack_predictions, y_true)

model_names = ["Decision Tree", "Stacking"]
bias_values = [tree_bias, stack_bias]
variance_values = [tree_variance, stack_variance]

display_results(model_names, bias_values, variance_values)

## Ансамблевые методы для задачи регрессии

Ансамблевые методы могут используются и в задачах регрессии. Однако, итоговая аппроксимация исходной зависимости имеет характер разбиения на регионы, внутри которых модель ведёт себя как некоторая простая функция – обычно постоянная или, при специальных модификациях, линейная.

Классические регрессионные деревья (например, `DecisionTreeRegressor`) разделяют пространство признаков на непересекающиеся регионы $ R_1, R_2, \dots, R_n $. В каждом регионе дерево присваивает константное значение, как правило, равное среднему значению целевой переменной для объектов, попавших в этот регион:
$$
f(x) = \sum_{i=1}^{n} c_i \cdot \mathbb{I}(x \in R_i)
$$
где:
- $ c_i $ — среднее значение отклика для $ R_i $,
- $\mathbb{I}(x \in R_i)$ — индикатор, равный 1, если $x$ принадлежит $R_i$.

Таким образом, восстановленная функция является **кусочно постоянной**.

#### Итог
Таким образом, **если базовой моделью является дерево решений с константным прогнозом в листьях**, восстановленная функция будет иметь вид кусочно постоянного приближения. Если же в листьях обучаются линейные модели, итоговая функция становится кусочно-линейной. Ансамблевые методы, усредняя предсказания базовых моделей, могут сгладить эти переходы, но принцип регионального разбиения пространства сохраняется.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor


np.random.seed(42)
n_samples = 10
X = np.random.uniform(0, 2*np.pi, n_samples)
X = np.sort(X)
y_true = true_function(X)
noise = np.random.normal(0, 0.2, n_samples)
y = y_true + noise

X = X.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = XGBRegressor(n_estimators=100,
                        random_state=42,
                        objective="reg:squarederror",
                        verbosity=0)

rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

y_test_pred_rf = rf_model.predict(X_test)
y_test_pred_gb = gb_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_test_pred_rf)
mse_gb = mean_squared_error(y_test, y_test_pred_gb)
print(f"MSE (Random Forest Regressor): {mse_rf:.2f}")
print(f"MSE (XGBoost Regressor): {mse_gb:.2f}")

x_grid = np.linspace(0, 2*np.pi, 500).reshape(-1, 1)
y_grid_true = true_function(x_grid.ravel())
y_grid_rf = rf_model.predict(x_grid)
y_grid_gb = gb_model.predict(x_grid)

plt.figure(figsize=(8, 6))

plt.plot(x_grid, y_grid_true, color='black', lw=2, label='Истинная функция')
plt.plot(x_grid, y_grid_rf, lw=2, label=f'Random Forest')
plt.plot(x_grid, y_grid_gb, lw=2, label=f'XGBoost')
plt.scatter(X, y, color='gray', alpha=0.6, label='Обучающие данные')
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()